In [5]:
import array

import random
import numpy as np

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

import operator
import math

import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout

import networkx as nx

In [6]:
def draw_tree(expr):
    nodes, edges, labels = gp.graph(expr)
    g = nx.Graph()
    g.add_nodes_from(nodes)
    g.add_edges_from(edges)
    pos = graphviz_layout(g, prog="dot")

    nx.draw_networkx_nodes(g, pos)
    nx.draw_networkx_edges(g, pos)
    nx.draw_networkx_labels(g, pos, labels)
    plt.show()

In [7]:

class RegexObject(object):
    index = 0
    def __init__(self):
        self.index = RegexObject.index
        RegexObject.index += 1
        self.next = None
        self.value = ""
        
    def add_next(self, next_object):
        self.next = next_object
        return self
    
    @staticmethod
    def create():
        return RegexObject()
    
    @staticmethod
    def create_with_value(value):
        regex = RegexObject()
        regex.value = value
        return regex
    
    @property
    def regex(self):
        result = self.value
        if self.next:
            result += self.next.regex 
        return result
     
class Group(RegexObject):
    def __init__(self):
        self.child = None

    def add_child(self, child, next_object):
        self.child = child
        return self.add_next(next_object)
    
    @property
    def group_value(self):
        if self.child.regex:
            return "(?:" + self.child.regex + ")+"
        return ""
        
    @property
    def regex(self):
        result = self.group_value
        if self.next:
            result += self.next.regex 
        return result
     
        
class GroupValue(RegexObject):
    pass

        
class Range(RegexObject):
    def __init__(self):
        self.child = None

    def add_child(self, child, next_object):
        self.child = child
        return self.add_next(next_object)
    
    @property
    def range_value(self):
        if self.child.regex:
            return "[" + self.child.regex + "]+"
        return ""
        
    @property
    def regex(self):
        result = self.range_value
        if self.next:
            result += self.next.regex 
        return result

    
class RangeValue(RegexObject):
    pass


    
class Factory(object):
    def __init__(self, value):
        self.value = ""
        
    @staticmethod
    def create_value(value):
        return lambda x:RegexObject.create_with_value(value).add_next(x)
    
    @staticmethod
    def create_group():
        return lambda x, y:Group().add_child(x, y)
    
    @staticmethod
    def create_range():
        return lambda x, y:Range().add_child(x, y)

    

In [8]:
pset = gp.PrimitiveSetTyped("main", [], RegexObject)

pset.addPrimitive(Factory.create_value("A"), [RegexObject], RegexObject, "RegexObject_A")
pset.addPrimitive(Factory.create_value("B"), [RegexObject], RegexObject, "RegexObject_B")
pset.addPrimitive(Factory.create_value("C"), [RegexObject], RegexObject, "RegexObject_C")
pset.addPrimitive(Factory.create_value("D"), [RegexObject], RegexObject, "RegexObject_D")
pset.addPrimitive(Factory.create_value("E"), [RegexObject], RegexObject, "RegexObject_E")

pset.addPrimitive(Factory.create_group(), [GroupValue, RegexObject], RegexObject, "Group")
pset.addPrimitive(Factory.create_value("A"), [RegexObject], GroupValue, "GroupValue_A")
pset.addPrimitive(Factory.create_value("B"), [RegexObject], GroupValue, "GroupValue_B")
pset.addPrimitive(Factory.create_value("C"), [RegexObject], GroupValue, "GroupValue_C")
pset.addPrimitive(Factory.create_value("D"), [RegexObject], GroupValue, "GroupValue_D")
pset.addPrimitive(Factory.create_value("E"), [RegexObject], GroupValue, "GroupValue_E")

pset.addPrimitive(Factory.create_range(), [RangeValue, RegexObject], RegexObject, "Range")
pset.addPrimitive(Factory.create_value("A"), [RangeValue], RangeValue, "RangeValue_A")
pset.addPrimitive(Factory.create_value("B"), [RangeValue], RangeValue, "RangeValue_B")
pset.addPrimitive(Factory.create_value("C"), [RangeValue], RangeValue, "RangeValue_C")
pset.addPrimitive(Factory.create_value("D"), [RangeValue], RangeValue, "RangeValue_D")
pset.addPrimitive(Factory.create_value("E"), [RangeValue], RangeValue, "RangeValue_E")

pset.addTerminal(RegexObject.create(), RegexObject, "RegexEnd")
pset.addTerminal(GroupValue.create(), GroupValue, "GroupEnd")
pset.addTerminal(RangeValue.create(), RangeValue, "RangeEnd")


In [9]:
creator.create("Fitness", base.Fitness, weights=(1.0, 1.0, -1.0, -1.0, -1.0))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.Fitness)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genFull, pset=pset, min_=5, max_=10)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

In [4]:
expr = toolbox.individual()

draw_tree(expr)

NameError: name 'toolbox' is not defined

In [7]:
tree = gp.PrimitiveTree(expr)
print gp.compile(tree, pset).regex

ACCEBEBDDC


In [1]:
PERFECT_SCALE = 1.0

def sign(x):
    if x > 0:
        return 1.0
    if x < 0:
        return -1.0
    return 0.0

import re, sys
correct_testset = ["ADEADEADE", "ADDEADDEE", "AAADDEEEAADDEEADE", "ADEADEEE","ADEEADEEEADEAAADE", "AADDEEAADDEE", "AADDEE", "AADDEE", "ADEEE", "AAADDE", "AAAAAADEEEE", "AAAADDDDEE", "ADDDDDEEEE", "AAADDDDE"]
wrong_testset = ["DDAA", "DEEA", "AAAEEEDDD", "DDDAAAEEE", "EEEEDDAA", "EDA", "EAAA", "DEDEDE", "AAAAA", "ADEDEDEDEDE", "ADEDEEEE", "EE", "AEE", "AEEEE", "AEEEED"]

def test_testset(regex, testset):
    total, partial, count = 0, 0, 0
    
    for test_item in testset:
        match = re.match(regex, test_item)
        if match:
            result = match.groups()[0]
            value = len(result) / float(len(test_item))
            if value == 1.0:
                total += 1
            partial += value
        count += 1.0

    if count == 0:
        return 0, 0
    return total, partial
            

def evaluate(individual):
    tree = gp.PrimitiveTree(individual)
    regex = "({})".format(gp.compile(tree, pset).regex)
    length = len(regex)**2
    if length == 0:
        length = sys.maxint
        
    correct, correct_partial = test_testset(regex, correct_testset)
    wrong, wrong_partial = test_testset(regex, wrong_testset)
    
    return correct, correct_partial, wrong, wrong_partial, length

In [2]:
toolbox.register("evaluate", evaluate)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=1, max_=20)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

stats_correct = tools.Statistics(lambda ind: ind.fitness.values[0])
stats_correct_partial = tools.Statistics(lambda ind: ind.fitness.values[1])
stats_wrong = tools.Statistics(lambda ind: ind.fitness.values[2])
stats_wrong_partial = tools.Statistics(lambda ind: ind.fitness.values[3])
stats_size = tools.Statistics(lambda ind: ind.fitness.values[4])

mstats = tools.MultiStatistics(stats_correct_partial=stats_correct_partial,
                               stats_wrong_partial=stats_wrong_partial,
                               stats_size=stats_size)
mstats.register("avg", np.mean)
mstats.register("max", np.max)

NameError: name 'toolbox' is not defined

In [56]:
NGEN = 20
pop = toolbox.population(n=5000)
hof = tools.HallOfFame(10)
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=NGEN, stats=mstats,
                               halloffame=hof, verbose=True)


   	      	 stats_correct_partial 	 stats_size 	  stats_wrong_partial   
   	      	-----------------------	------------	------------------------
gen	nevals	avg       	max       	avg    	max 	avg        	max     
0  	5000  	0.00089123	3.27968   	201.822	3364	0.000166667	0.833333
1  	3049  	0.00764302	4.80909   	133.428	7569	0.0133021  	2.83333 
2  	3004  	0.0279071 	4.80909   	115.072	7744	0.0284403  	2.1     
3  	2981  	0.0816259 	4.80909   	90.1402	5929	0.0584883  	2.1     
4  	3009  	0.200577  	10.4854   	86.1452	10816	0.115307   	6.49242 
5  	2977  	0.460404  	14        	90.768 	6241 	0.2375     	15      
6  	3032  	0.861701  	14        	115.445	13689	0.416983   	15      
7  	3017  	1.49999   	14        	157.175	12321	0.676391   	15      
8  	3019  	2.38293   	14        	235.64 	5929 	0.957204   	15      
9  	2962  	3.64865   	14        	394.382	5184 	1.39356    	15      
10 	3074  	5.19622   	14        	532.202	5041 	2.29328    	15      
11 	3023  	7.26503   	14        	558.224	10

In [58]:
for item in hof.items:
    tree = gp.PrimitiveTree(item)
    #draw_tree(tree)
    print gp.compile(tree, pset).regex, toolbox.evaluate(tree)

A[DEA]+E (14, 14.0, 4, 5.499999999999999, 100)
A[CDAE]+E (14, 14.0, 4, 5.499999999999999, 121)
A[CDDAE]+E (14, 14.0, 4, 5.499999999999999, 144)
A[CEDABE]+E (14, 14.0, 4, 5.499999999999999, 169)
A[CEDACA]+E (14, 14.0, 4, 5.499999999999999, 169)
A[CEDAEDE]+E (14, 14.0, 4, 5.499999999999999, 196)
A[CABEECD]+E (14, 14.0, 4, 5.499999999999999, 196)
A[CEDACAE]+E (14, 14.0, 4, 5.499999999999999, 196)
A[ADEBEDE]+E (14, 14.0, 4, 5.499999999999999, 196)
A[DCDEDBDA]+E (14, 14.0, 4, 5.499999999999999, 225)


In [ ]:
gp.compile(tree, pset).regex

In [ ]:
 for item in items:
    print re.match('((?:A)+(?:D)+(?:E)+)', item).groups()
    print best
    current_value = len(best) / float(len(item))
    #print current_value
    if current_value == 1.0:
        current_value *= PERFECT_SCALE
    print current_value
